In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import matplotlib.cm as cm



In [2]:
pd.set_option('display.max_columns', 35)

In [3]:
SHEET_ID = '1gzQTlP9P4LtWXPgZOOAPsDsxAYbO70eCcHxyHmsl794'
DATA_TAB = 'DATA'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
df = pd.read_csv(URL)


df.columns = ['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ', 'H1 ', 'H2 ', 'Color ']

df.drop(['Color ', 'H1 ', 'H2 '], axis=1, inplace=True)
# filter out match NaN
df = df[df['Match '].notna()]
df['Match '] = df['Match '].astype(int)

df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

# for each match number create cummulative sum of wins, score1 and score2 for each team
df_full1 = df_full.copy()
df_full1 = df_full1.sort_values(['Match ', 'Team1 '])


df_full1['WIN_CUMSUM'] = df_full1.groupby(['Team1 '])['WIN'].cumsum()
df_full1['SCORE1_CUMSUM'] = df_full1.groupby(['Team1 '])['Score1 '].cumsum()
df_full1['SCORE2_CUMSUM'] = df_full1.groupby(['Team1 '])['Score2 '].cumsum()

# select only the columns we need
df_full1 = df_full1[['Match ', 'Team1 ', 'WIN_CUMSUM', 'SCORE1_CUMSUM', 'SCORE2_CUMSUM']]

# create rank column within each week sorting by WIN_CUMSUM and then SCORE1_CUMSUM
df_full1 = df_full1.sort_values(['Match ', 'WIN_CUMSUM', 'SCORE1_CUMSUM'], ascending=[True, False, False])
df_full1['ranker'] = df_full1['WIN_CUMSUM']*1000000 + df_full1['SCORE1_CUMSUM']
df_full1['RANK'] = df_full1.groupby(['Match '])['ranker'].rank(ascending=False, method='first')#.astype(int)

df_full1['RANK'] = df_full1['RANK'].astype(int)

# df_full1[df_full1['Team1 '] == 'Memphis Grizzlies'].plot(x='Match ', y='RANK', kind='line', figsize=(15,5), title='Rank of Memphis Grizzlies over time')

# create dict of rankings for each team each week
rankings = {}
for team in df_full1['Team1 '].unique():
    rankings[team] = list(df_full1[df_full1['Team1 '] == team].set_index('Match ')['RANK'])

In [4]:
df_full

,Team1,Score1,Team2,Score2,PERIOD,Match,Result,WINNER,LOSER,WIN
0,Golden State Warriors,200.00,Brooklyn Nets,195.50,Scoring Period 1,1,FINAL,Golden State Warriors,Brooklyn Nets,1.0
1,Portland Trail Blazers,166.25,Atlanta Hawks,169.50,Scoring Period 1,1,FINAL,Atlanta Hawks,Portland Trail Blazers,0.0
2,Houston Rockets,206.50,Philadelphia 76ers,166.75,Scoring Period 1,1,FINAL,Houston Rockets,Philadelphia 76ers,1.0
3,Oklahoma City Thunder,239.75,Minnesota Timberwolves,288.50,Scoring Period 1,1,FINAL,Minnesota Timberwolves,Oklahoma City Thunder,0.0
4,Toronto Raptors,213.50,Sacramento Kings,189.75,Scoring Period 1,1,FINAL,Toronto Raptors,Sacramento Kings,1.0
...,...,...,...,...,...,...,...,...,...,...
586,Oklahoma City Thunder,436.75,Phoenix Suns,483.25,Scoring Period 33,33,FINAL,Phoenix Suns,Oklahoma City Thunder,0.0
587,Cleveland Cavaliers,328.00,Boston Celtics,240.50,Scoring Period 33,33,FINAL,Cleveland Cavaliers,Boston Celtics,1.0
588,Miami Heat,208.25,Philadelphia 76ers,385.50,Scoring Period 33,33,FINAL,Philadelphia 76ers,Miami Heat,0.0
589,San Antonio Spurs,358.00,New York Knicks,414.00,Scoring Period 33,33,FINAL,New York Knicks,San Antonio Spurs,0.0


In [5]:
df_scores = df_full.groupby(['Team1 '])[['Score1 ', 'Score2 ']].sum().reset_index()

df_scores.columns = ['Team', 'Scored For', 'Scored Against']
median_scored_for = np.median(df_scores['Scored For'])
median_scored_against = np.median(df_scores['Scored Against'])

In [6]:
df_scores

,Team,Scored For,Scored Against
0,Atlanta Hawks,12342.75,11932.50
1,Boston Celtics,10967.75,11838.25
2,Brooklyn Nets,12458.00,11618.00
3,Charlotte Hornets,14376.75,12697.75
4,Chicago Bulls,11820.50,12682.00
5,Cleveland Cavaliers,9737.75,11506.25
6,Dallas Mavericks,14097.50,11999.25
7,Denver Nuggets,13513.75,11979.75
8,Detroit Pistons,13322.50,12536.25
9,Golden State Warriors,12778.75,11348.25


In [7]:
rankings


{'Chicago Bulls': [1,
  14,
  7,
  5,
  7,
  6,
  9,
  11,
  14,
  13,
  17,
  18,
  17,
  16,
  16,
  16,
  16,
  14,
  14,
  15,
  15,
  15,
  16,
  17,
  18,
  18,
  18,
  19,
  19,
  19,
  19,
  19,
  20],
 'Dallas Mavericks': [2,
  4,
  6,
  9,
  14,
  17,
  14,
  15,
  13,
  11,
  10,
  12,
  12,
  11,
  8,
  11,
  12,
  10,
  10,
  7,
  10,
  9,
  11,
  9,
  9,
  7,
  7,
  6,
  4,
  6,
  4,
  6,
  8],
 'Utah Jazz': [3,
  2,
  3,
  3,
  3,
  3,
  2,
  2,
  3,
  5,
  6,
  5,
  7,
  12,
  10,
  12,
  9,
  11,
  11,
  8,
  6,
  10,
  8,
  10,
  10,
  8,
  8,
  7,
  5,
  7,
  5,
  7,
  5],
 'Minnesota Timberwolves': [4,
  6,
  4,
  4,
  4,
  5,
  5,
  4,
  7,
  6,
  4,
  7,
  9,
  7,
  12,
  8,
  10,
  15,
  15,
  16,
  16,
  16,
  17,
  16,
  16,
  16,
  15,
  15,
  13,
  12,
  13,
  12,
  10],
 'Charlotte Hornets': [5,
  9,
  15,
  12,
  9,
  7,
  6,
  5,
  4,
  7,
  8,
  8,
  6,
  8,
  5,
  4,
  3,
  6,
  5,
  5,
  5,
  1,
  1,
  4,
  5,
  5,
  6,
  5,
  8,
  5,
  7,
  5,
  4],
 '

In [8]:
# for each match number create cummulative sum of wins, score1 and score2 for each team
df_full1 = df_full.copy()
df_full1 = df_full1.sort_values(['Match ', 'Team1 '])

df_full1['WIN_CUMSUM'] = df_full1.groupby(['Team1 '])['WIN'].cumsum()
df_full1['SCORE1_CUMSUM'] = df_full1.groupby(['Team1 '])['Score1 '].cumsum()
df_full1['SCORE2_CUMSUM'] = df_full1.groupby(['Team1 '])['Score2 '].cumsum()

# select only the columns we need
df_full1 = df_full1[['Match ', 'Team1 ', 'WIN_CUMSUM', 'SCORE1_CUMSUM', 'SCORE2_CUMSUM']]

# create rank column within each week sorting by WIN_CUMSUM and then SCORE1_CUMSUM
df_full1 = df_full1.sort_values(['Match ', 'WIN_CUMSUM', 'SCORE1_CUMSUM'], ascending=[True, False, False])
df_full1['ranker'] = df_full1['WIN_CUMSUM'] * 1000000 + df_full1['SCORE1_CUMSUM']
df_full1['RANK'] = df_full1.groupby(['Match '])['ranker'].rank(ascending=False, method='first')  # .astype(int)

df_full1['RANK'] = df_full1['RANK'].astype(int)

# create dict of rankings for each team each week
rankings = {}
for team in df_full1['Team1 '].unique():
    rankings[team] = list(df_full1[df_full1['Team1 '] == team].set_index('Match ')['RANK'])

In [9]:
SHEET_ID = '1gzQTlP9P4LtWXPgZOOAPsDsxAYbO70eCcHxyHmsl794'
DATA_TAB = 'DATA'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
df = pd.read_csv(URL)


df.columns = ['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ', 'H1 ', 'H2 ', 'Color ']

df.drop(['Color ', 'H1 ', 'H2 '], axis=1, inplace=True)
# filter out match NaN
df = df[df['Match '].notna()]
df['Match '] = df['Match '].astype(int)

df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

In [10]:
df_full

,Team1,Score1,Team2,Score2,PERIOD,Match,Result,WINNER,LOSER,WIN
0,Golden State Warriors,200.00,Brooklyn Nets,195.50,Scoring Period 1,1,FINAL,Golden State Warriors,Brooklyn Nets,1.0
1,Portland Trail Blazers,166.25,Atlanta Hawks,169.50,Scoring Period 1,1,FINAL,Atlanta Hawks,Portland Trail Blazers,0.0
2,Houston Rockets,206.50,Philadelphia 76ers,166.75,Scoring Period 1,1,FINAL,Houston Rockets,Philadelphia 76ers,1.0
3,Oklahoma City Thunder,239.75,Minnesota Timberwolves,288.50,Scoring Period 1,1,FINAL,Minnesota Timberwolves,Oklahoma City Thunder,0.0
4,Toronto Raptors,213.50,Sacramento Kings,189.75,Scoring Period 1,1,FINAL,Toronto Raptors,Sacramento Kings,1.0
...,...,...,...,...,...,...,...,...,...,...
586,Oklahoma City Thunder,436.75,Phoenix Suns,483.25,Scoring Period 33,33,FINAL,Phoenix Suns,Oklahoma City Thunder,0.0
587,Cleveland Cavaliers,328.00,Boston Celtics,240.50,Scoring Period 33,33,FINAL,Cleveland Cavaliers,Boston Celtics,1.0
588,Miami Heat,208.25,Philadelphia 76ers,385.50,Scoring Period 33,33,FINAL,Philadelphia 76ers,Miami Heat,0.0
589,San Antonio Spurs,358.00,New York Knicks,414.00,Scoring Period 33,33,FINAL,New York Knicks,San Antonio Spurs,0.0


In [11]:
df_alter = df_full.copy()

#select only the columns we need
df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']
df_alter


,Match,Alt_Team,Alt_Score
0,1,Golden State Warriors,200.00
1,1,Portland Trail Blazers,166.25
2,1,Houston Rockets,206.50
3,1,Oklahoma City Thunder,239.75
4,1,Toronto Raptors,213.50
...,...,...,...
586,33,Oklahoma City Thunder,436.75
587,33,Cleveland Cavaliers,328.00
588,33,Miami Heat,208.25
589,33,San Antonio Spurs,358.00


In [12]:
df_full.columns

Index(['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ',
       'Result ', 'WINNER ', 'LOSER ', 'WIN'],
      dtype='object')

In [13]:
df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')

# replace 0 with -1
# df_wins = df_wins.replace(0, -1)
# df_wins = df_wins.replace(0.5, 0)

#fill nan with 0
df_wins = df_wins.fillna(-1)

df_wins


Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,-1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
Boston Celtics,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Brooklyn Nets,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
Charlotte Hornets,0.0,1.0,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0
Chicago Bulls,0.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Cleveland Cavaliers,0.0,2.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Dallas Mavericks,1.0,1.0,0.0,1.0,0.0,1.0,-1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
Denver Nuggets,0.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
Detroit Pistons,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [14]:
df_alt_merge = df_full.merge(df_alter, on='Match ', how = 'inner')
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])

# remove rows where Team1 = Alt_Team
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]

#filter by team Memphis Grizzlies and match number = 1
# df_alt_merge[(df_alt_merge['Team1 '] == 'Memphis Grizzlies') & (df_alt_merge['Match '] == 1)]

df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

#divide by number of games played between each team
df_alt_wins = df_alt_wins.div(pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='count'), axis=1)

#round up to 2 decimal places
df_alt_wins = df_alt_wins.round(2)


df_alt_wins




Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,0.70,0.58,0.33,0.55,0.73,0.36,0.39,0.48,0.48,0.58,0.27,0.76,0.70,0.39,0.88,0.97,0.42,0.42,0.79,0.61,0.30,0.91,0.64,0.42,0.39,0.76,0.36,0.39,0.50
Boston Celtics,0.30,NaN,0.36,0.06,0.42,0.55,0.21,0.27,0.24,0.24,0.36,0.12,0.67,0.61,0.18,0.82,0.97,0.24,0.15,0.61,0.52,0.12,0.79,0.39,0.12,0.27,0.67,0.15,0.15,0.30
Brooklyn Nets,0.42,0.64,NaN,0.27,0.52,0.76,0.27,0.42,0.36,0.44,0.48,0.38,0.85,0.76,0.30,0.88,1.00,0.33,0.30,0.82,0.61,0.24,0.85,0.55,0.30,0.42,0.79,0.33,0.38,0.52
Charlotte Hornets,0.67,0.94,0.73,NaN,0.70,0.94,0.55,0.55,0.73,0.64,0.85,0.52,1.00,0.91,0.61,1.00,1.00,0.55,0.58,0.94,0.94,0.39,1.00,0.76,0.61,0.58,1.00,0.73,0.55,0.70
Chicago Bulls,0.45,0.58,0.48,0.30,NaN,0.70,0.36,0.24,0.36,0.39,0.52,0.21,0.76,0.64,0.24,0.73,0.95,0.33,0.30,0.76,0.55,0.24,0.73,0.52,0.36,0.30,0.82,0.39,0.24,0.42
Cleveland Cavaliers,0.27,0.45,0.24,0.06,0.30,NaN,0.09,0.27,0.18,0.12,0.33,0.09,0.52,0.48,0.06,0.73,0.94,0.15,0.09,0.61,0.36,0.12,0.67,0.33,0.15,0.15,0.61,0.15,0.15,0.21
Dallas Mavericks,0.64,0.79,0.73,0.45,0.64,0.91,NaN,0.61,0.58,0.61,0.70,0.55,0.91,0.82,0.52,0.91,1.00,0.58,0.58,0.91,0.73,0.42,0.86,0.82,0.55,0.45,0.91,0.58,0.36,0.52
Denver Nuggets,0.61,0.73,0.58,0.45,0.76,0.73,0.39,NaN,0.52,0.70,0.76,0.45,0.79,0.76,0.45,0.88,0.94,0.52,0.45,0.85,0.73,0.42,0.79,0.70,0.52,0.48,0.92,0.61,0.42,0.64
Detroit Pistons,0.52,0.76,0.64,0.27,0.64,0.82,0.42,0.48,NaN,0.58,0.70,0.42,0.91,0.79,0.42,0.91,1.00,0.48,0.45,0.85,0.76,0.33,0.85,0.76,0.36,0.61,0.91,0.53,0.42,0.61


In [15]:
df_alt_wins['Strength'] = df_alt_wins.mean(axis=1)

# add rank column
df_alt_wins['Rank'] = df_alt_wins['Strength'].rank(ascending=False).astype(int)

# Fill NaN values with -1 to indicate "not applicable"
df_alt_wins = df_alt_wins.fillna(-1)

# Sort teams alphabetically
teams = sorted([col for col in df_alt_wins.columns if col not in ['Strength', 'Rank']])

# Reorder columns to match sorted teams (plus Strength and Rank at the end)
df_alt_wins = df_alt_wins[teams + ['Strength', 'Rank']]

In [16]:
df_alt_wins

Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards,Strength,Rank
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,-1.00,0.70,0.58,0.33,0.55,0.73,0.36,0.39,0.48,0.48,0.58,0.27,0.76,0.70,0.39,0.88,0.97,0.42,0.42,0.79,0.61,0.30,0.91,0.64,0.42,0.39,0.76,0.36,0.39,0.50,0.553793,15
Boston Celtics,0.30,-1.00,0.36,0.06,0.42,0.55,0.21,0.27,0.24,0.24,0.36,0.12,0.67,0.61,0.18,0.82,0.97,0.24,0.15,0.61,0.52,0.12,0.79,0.39,0.12,0.27,0.67,0.15,0.15,0.30,0.374483,22
Brooklyn Nets,0.42,0.64,-1.00,0.27,0.52,0.76,0.27,0.42,0.36,0.44,0.48,0.38,0.85,0.76,0.30,0.88,1.00,0.33,0.30,0.82,0.61,0.24,0.85,0.55,0.30,0.42,0.79,0.33,0.38,0.52,0.523793,17
Charlotte Hornets,0.67,0.94,0.73,-1.00,0.70,0.94,0.55,0.55,0.73,0.64,0.85,0.52,1.00,0.91,0.61,1.00,1.00,0.55,0.58,0.94,0.94,0.39,1.00,0.76,0.61,0.58,1.00,0.73,0.55,0.70,0.747241,2
Chicago Bulls,0.45,0.58,0.48,0.30,-1.00,0.70,0.36,0.24,0.36,0.39,0.52,0.21,0.76,0.64,0.24,0.73,0.95,0.33,0.30,0.76,0.55,0.24,0.73,0.52,0.36,0.30,0.82,0.39,0.24,0.42,0.478276,18
Cleveland Cavaliers,0.27,0.45,0.24,0.06,0.30,-1.00,0.09,0.27,0.18,0.12,0.33,0.09,0.52,0.48,0.06,0.73,0.94,0.15,0.09,0.61,0.36,0.12,0.67,0.33,0.15,0.15,0.61,0.15,0.15,0.21,0.306207,24
Dallas Mavericks,0.64,0.79,0.73,0.45,0.64,0.91,-1.00,0.61,0.58,0.61,0.70,0.55,0.91,0.82,0.52,0.91,1.00,0.58,0.58,0.91,0.73,0.42,0.86,0.82,0.55,0.45,0.91,0.58,0.36,0.52,0.677241,6
Denver Nuggets,0.61,0.73,0.58,0.45,0.76,0.73,0.39,-1.00,0.52,0.70,0.76,0.45,0.79,0.76,0.45,0.88,0.94,0.52,0.45,0.85,0.73,0.42,0.79,0.70,0.52,0.48,0.92,0.61,0.42,0.64,0.639655,9
Detroit Pistons,0.52,0.76,0.64,0.27,0.64,0.82,0.42,0.48,-1.00,0.58,0.70,0.42,0.91,0.79,0.42,0.91,1.00,0.48,0.45,0.85,0.76,0.33,0.85,0.76,0.36,0.61,0.91,0.53,0.42,0.61,0.627586,11


In [17]:

SHEET_ID = '1gzQTlP9P4LtWXPgZOOAPsDsxAYbO70eCcHxyHmsl794'
DATA_TAB = 'DATA'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
df = pd.read_csv(URL)


df.columns = ['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ', 'H1 ', 'H2 ', 'Color ']

df.drop(['Color ', 'H1 ', 'H2 '], axis=1, inplace=True)
# filter out match NaN
df = df[df['Match '].notna()]
df['Match '] = df['Match '].astype(int)

df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')

df_alter = df_full.copy()

#select only the columns we need
df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']
df_alter


df_alt_merge = df_full.merge(df_alter, on='Match ', how = 'inner')
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])

# remove rows where Team1 = Alt_Team
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]


df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

#divide by number of games played between each team
df_alt_wins = df_alt_wins.div(pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='count'), axis=1)

#round up to 2 decimal places
df_alt_wins = df_alt_wins.round(2)

In [18]:
df_wins

Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
Boston Celtics,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Brooklyn Nets,2.0,1.0,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
Charlotte Hornets,0.0,1.0,1.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0
Chicago Bulls,0.0,1.0,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Cleveland Cavaliers,0.0,2.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Dallas Mavericks,1.0,1.0,0.0,1.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
Denver Nuggets,0.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,2.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
Detroit Pistons,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [19]:
df_alt_wins

Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,0.70,0.58,0.33,0.55,0.73,0.36,0.39,0.48,0.48,0.58,0.27,0.76,0.70,0.39,0.88,0.97,0.42,0.42,0.79,0.61,0.30,0.91,0.64,0.42,0.39,0.76,0.36,0.39,0.50
Boston Celtics,0.30,NaN,0.36,0.06,0.42,0.55,0.21,0.27,0.24,0.24,0.36,0.12,0.67,0.61,0.18,0.82,0.97,0.24,0.15,0.61,0.52,0.12,0.79,0.39,0.12,0.27,0.67,0.15,0.15,0.30
Brooklyn Nets,0.42,0.64,NaN,0.27,0.52,0.76,0.27,0.42,0.36,0.44,0.48,0.38,0.85,0.76,0.30,0.88,1.00,0.33,0.30,0.82,0.61,0.24,0.85,0.55,0.30,0.42,0.79,0.33,0.38,0.52
Charlotte Hornets,0.67,0.94,0.73,NaN,0.70,0.94,0.55,0.55,0.73,0.64,0.85,0.52,1.00,0.91,0.61,1.00,1.00,0.55,0.58,0.94,0.94,0.39,1.00,0.76,0.61,0.58,1.00,0.73,0.55,0.70
Chicago Bulls,0.45,0.58,0.48,0.30,NaN,0.70,0.36,0.24,0.36,0.39,0.52,0.21,0.76,0.64,0.24,0.73,0.95,0.33,0.30,0.76,0.55,0.24,0.73,0.52,0.36,0.30,0.82,0.39,0.24,0.42
Cleveland Cavaliers,0.27,0.45,0.24,0.06,0.30,NaN,0.09,0.27,0.18,0.12,0.33,0.09,0.52,0.48,0.06,0.73,0.94,0.15,0.09,0.61,0.36,0.12,0.67,0.33,0.15,0.15,0.61,0.15,0.15,0.21
Dallas Mavericks,0.64,0.79,0.73,0.45,0.64,0.91,NaN,0.61,0.58,0.61,0.70,0.55,0.91,0.82,0.52,0.91,1.00,0.58,0.58,0.91,0.73,0.42,0.86,0.82,0.55,0.45,0.91,0.58,0.36,0.52
Denver Nuggets,0.61,0.73,0.58,0.45,0.76,0.73,0.39,NaN,0.52,0.70,0.76,0.45,0.79,0.76,0.45,0.88,0.94,0.52,0.45,0.85,0.73,0.42,0.79,0.70,0.52,0.48,0.92,0.61,0.42,0.64
Detroit Pistons,0.52,0.76,0.64,0.27,0.64,0.82,0.42,0.48,NaN,0.58,0.70,0.42,0.91,0.79,0.42,0.91,1.00,0.48,0.45,0.85,0.76,0.33,0.85,0.76,0.36,0.61,0.91,0.53,0.42,0.61


In [20]:
# subtract df_alt_wins from df_wins
df_diff = df_wins - df_alt_wins

df_diff['Strength'] = df_diff.mean(axis=1)

# add rank column
df_diff['Rank'] = df_diff['Strength'].rank(ascending=False).astype(int)

# Fill NaN values with -1 to indicate "not applicable"
df_diff = df_diff.fillna(-1)

# Sort teams alphabetically
teams = sorted([col for col in df_diff.columns if col not in ['Strength', 'Rank']])

# Reorder columns to match sorted teams (plus Strength and Rank at the end)
df_diff = df_diff[teams + ['Strength', 'Rank']]

df_diff

Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards,Strength,Rank
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,-1.00,0.30,-0.58,0.67,0.45,0.27,-0.36,0.61,0.52,-0.48,-0.58,-0.27,0.24,0.30,0.61,0.12,0.03,-0.42,0.58,-0.79,-0.61,0.70,0.09,0.36,0.58,-0.39,0.24,-0.36,-0.39,0.50,0.066897,18
Boston Celtics,-0.30,-1.00,-0.36,-0.06,-0.42,-0.55,-0.21,-0.27,-0.24,-0.24,-0.36,-0.12,0.33,0.39,-0.18,0.18,0.03,-0.24,-0.15,0.39,0.48,0.88,1.21,0.61,-0.12,-0.27,-0.67,-0.15,-0.15,-0.30,-0.029655,28
Brooklyn Nets,1.58,0.36,-1.00,-0.27,-0.52,0.24,0.73,-0.42,-0.36,-0.44,1.52,0.62,-0.85,-0.76,0.70,0.12,0.00,-0.33,0.70,0.18,-0.61,-0.24,0.15,0.45,-0.30,0.58,0.21,-0.33,0.62,0.48,0.131379,5
Charlotte Hornets,-0.67,0.06,0.27,-1.00,-0.70,-0.94,0.45,-0.55,-0.73,-0.64,-0.85,0.48,0.00,0.09,0.39,0.00,0.00,0.45,0.42,0.06,0.06,0.61,0.00,-0.76,-0.61,0.42,0.00,1.27,1.45,0.30,0.011379,25
Chicago Bulls,-0.45,0.42,0.52,0.70,-1.00,0.30,0.64,-0.24,-0.36,-0.39,-0.52,0.79,0.24,0.36,-0.24,-0.73,0.05,-0.33,-0.30,0.24,0.45,-0.24,0.27,-0.52,-0.36,0.70,0.18,-0.39,-0.24,-0.42,0.004483,27
Cleveland Cavaliers,-0.27,1.55,-0.24,0.94,-0.30,-1.00,-0.09,-0.27,-0.18,-0.12,-0.33,-0.09,-0.52,0.52,-0.06,0.27,0.06,-0.15,-0.09,1.39,-0.36,-0.12,0.33,-0.33,-0.15,0.85,0.39,-0.15,-0.15,-0.21,0.073103,15
Dallas Mavericks,0.36,0.21,-0.73,0.55,-0.64,0.09,-1.00,0.39,-0.58,-0.61,0.30,-0.55,0.09,0.18,-0.52,0.09,0.00,0.42,0.42,0.09,0.27,0.58,0.14,0.18,0.45,0.55,-0.91,0.42,0.64,-0.52,0.046897,22
Denver Nuggets,-0.61,0.27,0.42,0.55,0.24,0.27,-0.39,-1.00,1.48,0.30,-0.76,0.55,0.21,0.24,-0.45,0.12,0.06,-0.52,-0.45,-0.85,0.27,0.58,0.21,0.30,-0.52,0.52,0.08,0.39,0.58,-0.64,0.084483,12
Detroit Pistons,-0.52,0.24,0.36,0.73,0.36,0.18,0.58,-0.48,-1.00,-0.58,-0.70,-0.42,0.09,0.21,-0.42,0.09,0.00,0.52,0.55,0.15,0.24,-0.33,-0.85,0.24,0.64,0.39,0.09,0.47,-0.42,0.39,0.062069,20


In [24]:
df_diff['Strength'].sum()

np.float64(2.0)

In [25]:
# sum of all values in df_diff per row except last two columns
df_diff.iloc[:, :-2].sum(axis=1)


Team1 
Atlanta Hawks             1.94
Boston Celtics           -0.86
Brooklyn Nets             3.81
Charlotte Hornets         0.33
Chicago Bulls             0.13
Cleveland Cavaliers       2.12
Dallas Mavericks          1.36
Denver Nuggets            2.45
Detroit Pistons           1.80
Golden State Warriors     3.38
Houston Rockets           4.23
Indiana Pacers            3.63
Los Angeles Clippers      3.02
Los Angeles Lakers        2.09
Memphis Grizzlies         4.53
Miami Heat                2.60
Milwaukee Bucks           0.15
Minnesota Timberwolves    2.37
New Orleans Pelicans     -2.67
New York Knicks           2.22
Oklahoma City Thunder     0.89
Orlando Magic             3.69
Philadelphia 76ers        3.85
Phoenix Suns              1.22
Portland Trail Blazers    2.83
Sacramento Kings          1.75
San Antonio Spurs        -1.22
Toronto Raptors           4.49
Utah Jazz                 1.86
Washington Wizards        2.01
dtype: float64

In [22]:
df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')
df_wins = df_wins.fillna(-1)

df_alter = df_full.copy()

# select only the columns we need
df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']

df_alt_merge = df_full.merge(df_alter, on='Match ', how = 'inner')
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])

# remove rows where Team1 = Alt_Team
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]

df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

# divide by number of games played between each team
df_alt_wins = df_alt_wins.div(pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='count'), axis=1)

# round up to 2 decimal places
df_alt_wins = df_alt_wins.round(2)

# Fill NaN values with -1 to indicate "not applicable"
df_alt_wins = df_alt_wins.fillna(-1)

# subtract df_alt_wins from df_wins
df_diff = df_wins - df_alt_wins

df_diff['Strength'] = df_diff.mean(axis=1)

# add rank column
df_diff['Rank'] = df_diff['Strength'].rank(ascending=False).astype(int)

# Fill NaN values with -1 to indicate "not applicable"
df_diff = df_diff.fillna(-1)

# Sort teams alphabetically
teams = sorted([col for col in df_diff.columns if col not in ['Strength', 'Rank']])

# Reorder columns to match sorted teams (plus Strength and Rank at the end)
df_diff = df_diff[teams + ['Strength', 'Rank']]

# Format the data for frontend consumption
matrix_data = df_diff.reset_index().to_dict('records')

matrix_data_wins = df_wins.reset_index().to_dict('records')

In [23]:
df_wins

Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,-1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
Boston Celtics,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Brooklyn Nets,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
Charlotte Hornets,0.0,1.0,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0
Chicago Bulls,0.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Cleveland Cavaliers,0.0,2.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Dallas Mavericks,1.0,1.0,0.0,1.0,0.0,1.0,-1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
Denver Nuggets,0.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
Detroit Pistons,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [24]:
import numpy as np

# Step 1: sampling grid
X_values = np.array([3, 4, 5, 6, 7, 8])
Y_values = np.linspace(-5, 5, 201)

# Prepare arrays to store (x_i, y_i) and the target f_i
x_data = []
y_data = []
f_data = []

# Step 2: compute F(X, Y) and collect data
for X in X_values:
    for Y in Y_values:
        F_val = ((X - 3) / 4.0) * np.tanh(X * Y)
        x_data.append(X)
        y_data.append(Y)
        f_data.append(F_val)

x_data = np.array(x_data)
y_data = np.array(y_data)
f_data = np.array(f_data)

# Step 3: Build design matrix
# Each row is [X, Y], corresponding to A*X + B*Y
M = np.column_stack((x_data, y_data))

# Step 4: Least-squares solution for A and B
# Solve M * [A, B]^T = f_data
params, residuals, rank, s = np.linalg.lstsq(M, f_data, rcond=None)
A_est, B_est = params

print("Estimated A =", A_est)
print("Estimated B =", B_est)


Estimated A = 7.88823465070619e-18
Estimated B = 0.1864140259717622


In [27]:
import numpy as np

def F(X, Y):
    return ((X - 3) / 4.0) * np.tanh(X * Y)

# 1. Create grid
X_values = np.array([3, 4, 5, 6, 7, 8])
Y_values = np.linspace(-1, 1, 201)  # 201 points in [-1,1]

# 2. Compute (x, y, f) data
x_data = []
y_data = []
f_data = []

for X in X_values:
    for Y in Y_values:
        x_data.append(X)
        y_data.append(Y)
        f_data.append(F(X, Y))

x_data = np.array(x_data)
y_data = np.array(y_data)
f_data = np.array(f_data)

# 3. Design matrix: each row = [X, Y]
M = np.column_stack((x_data, y_data))

# 4. Solve for [A, B] in A*X + B*Y ~ F(X,Y)
params, residuals, rank, s = np.linalg.lstsq(M, f_data, rcond=None)
A_est, B_est = params

print("A =", A_est)
print("B =", B_est)


A = 0.0
B = 0.9133381183606306


In [28]:
Y = 0.01

for X in X_values:
    print(X,  np.tanh(X * Y))

3 0.029991003238820143
4 0.03997868031116357
5 0.04995837495787998
6 0.059928103529143496
7 0.06988589031642899
8 0.07982976911113136


In [5]:
5*Y

3.5999999999999996

In [12]:
# Step 3 with intercept
M = np.column_stack((np.ones_like(x_data), x_data, y_data))

# Step 4
params, residuals, rank, s = np.linalg.lstsq(M, f_data, rcond=None)
C_est, A_est, B_est = params

print("C =", C_est, " A =", A_est, " B =", B_est)


C = -7.633203879574102e-17  A = 1.3548995448620395e-17  B = 0.9133381183606303


In [16]:
import numpy as np

def F(X, Y):
    return ((X - 3) / 4.0) * np.tanh(X * Y)

# Suppose you have a single Y of interest:
Y_interest = 0.5  # Example

# Discrete X values
X_values = [3, 4, 5, 6, 7, 8]

# Let's say the "true" X for this scenario is 5
X_true = 5

# Calculate F(X, Y_interest) for all possible X
F_at_Y = [F(X, Y_interest) for X in X_values]

# Look up the function value at the true X
F_true = F(X_true, Y_interest)

# Compare F_true against each X
for X in X_values:
    diff = F_true - F(X, Y_interest)
    print(f"X={X}: F({X_true},{Y_interest}) - F({X},{Y_interest}) = {diff:.4f}")


X=3: F(5,0.5) - F(3,0.5) = 0.4933
X=4: F(5,0.5) - F(4,0.5) = 0.2523
X=5: F(5,0.5) - F(5,0.5) = 0.0000
X=6: F(5,0.5) - F(6,0.5) = -0.2530
X=7: F(5,0.5) - F(7,0.5) = -0.5049
X=8: F(5,0.5) - F(8,0.5) = -0.7559


In [17]:
F(3, 0.5)

np.float64(0.0)

In [18]:
F(5, 0.5)

np.float64(0.49330714907571516)

In [21]:
import numpy as np

def F_func(X, Y):
    """
    Computes F(X,Y) = ((X - 3)/4)*tanh(X*Y).
    """
    return ((X - 3)/4.0) * np.tanh(X * Y)

# Example data (replace these with your actual arrays).
# Must have the same length for X_values and Y_values.
X_values = np.array([3, 4, 5, 6, 5, 7, 8, 3, 6, 7])
Y_values = np.array([0.2, 0.3, -0.7, 0.5, 0.5, -0.9, 1.0, -0.4, 0.3, 0.99])

# 1. Compute F_data for each (X, Y)
F_data = np.array([F_func(x, y) for x, y in zip(X_values, Y_values)])

# 2. Build design matrix for linear model without intercept: A*X + B*Y
M = np.column_stack((X_values, Y_values))

# 3. Solve M * [A, B]^T ≈ F_data
params, residuals, rank, s = np.linalg.lstsq(M, F_data, rcond=None)
A_est, B_est = params

# 4. Print results
print("Estimated coefficient A =", A_est)
print("Estimated coefficient B =", B_est)

# (Optional) Compute and show the root mean squared error
preds = M.dot(params)
rms_error = np.sqrt(np.mean((F_data - preds)**2))
print("Root Mean Squared Error =", rms_error)


Estimated coefficient A = 0.020959206088787777
Estimated coefficient B = 0.9883440589478584
Root Mean Squared Error = 0.20404635061408238


In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Coaching career data with example logos
data = [
    {"Club": "Ilisiakos", "Country": "Greece", "Start": "2002-07-01", "End": "2004-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/38/Ilisiakos_logo.png"},
    {"Club": "AEK Athens", "Country": "Greece", "Start": "2004-07-01", "End": "2005-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/thumb/f/f4/AEK_Athens_FC.svg/1200px-AEK_Athens_FC.svg.png"},
    {"Club": "Larissa", "Country": "Greece", "Start": "2007-01-01", "End": "2008-12-31", "Logo": "https://upload.wikimedia.org/wikipedia/en/9/92/AE_Larissa_logo.svg"},
    {"Club": "Atromitos", "Country": "Greece", "Start": "2009-01-01", "End": "2012-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/8/8c/Atromitos_FC_logo.svg"},
    {"Club": "PAOK", "Country": "Greece", "Start": "2012-07-01", "End": "2013-04-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/3e/PAOK_FC_logo.svg"},
    {"Club": "APOEL", "Country": "Cyprus", "Start": "2013-05-01", "End": "2015-01-06", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/3c/APOEL_FC.svg"},
    {"Club": "Al-Hilal", "Country": "Saudi Arabia", "Start": "2015-05-25", "End": "2016-02-21", "Logo": "https://upload.wikimedia.org/wikipedia/en/4/44/Al-Hilal_FC_logo.svg"},
    {"Club": "Sharjah", "Country": "UAE", "Start": "2017-01-01", "End": "2017-06-01", "Logo": "https://upload.wikimedia.org/wikipedia/en/b/b1/Sharjah_FC_Logo.png"},
    {"Club": "Panathinaikos", "Country": "Greece", "Start": "2018-05-19", "End": "2020-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/e/eb/Panathinaikos_FC_logo.svg"},
    {"Club": "Maccabi Tel Aviv", "Country": "Israel", "Start": "2020-10-28", "End": "2021-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/f/fb/Maccabi_tel_aviv_fc.svg"},
    {"Club": "Al-Wehda", "Country": "Saudi Arabia", "Start": "2021-07-01", "End": "2022-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/33/Al-Wehda_Club_logo.svg"},
]

df = df.sort_values("Start")
df["Midpoint"] = df["Start"] + (df["End"] - df["Start"]) / 2

fig = px.timeline(
    df,
    x_start="Start",
    x_end="End",
    y="Club",
    color="Country",
    hover_data=["Country", "Start", "End"],
    category_orders={"Club": df["Club"].tolist()}  # sort y-axis by start date
)

fig.update_yaxes(autorange="reversed")

# Add logos as images
for _, row in df.iterrows():
    fig.add_layout_image(
        dict(
            source=row["Logo"],
            x=row["Midpoint"],
            y=row["Club"],
            xref="x",
            yref="y",
            sizex=(row["End"] - row["Start"]).days / 5,
            sizey=0.4,
            xanchor="center",
            yanchor="middle",
            sizing="contain",
            layer="above"
        )
    )

fig.update_layout(
    template="plotly_dark",
    title="👔 Georgios Donis – Coaching Career Timeline",
    title_font_size=26,
    font=dict(family="Helvetica Neue", size=14),
    height=700,
    margin=dict(l=70, r=70, t=100, b=50),
    plot_bgcolor="#111",
    paper_bgcolor="#111",
    legend_title="",
    hoverlabel=dict(bgcolor="#222", font_size=13, font_family="Arial")
)

fig.show()


In [26]:
# Log Google Sheets URL construction
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
# logger.info(f"Fetching data from Google Sheets URL: {URL}")

# Fetch data with error handling

df = pd.read_csv(URL)
# logger.info("Successfully fetched CSV data from Google Sheets")
# log_data_shape(logger, df, "initial_fetch", "raw CSV data")

# Drop unnecessary columns
columns_to_drop = ['Color ', 'H1 ', 'H2 ']
df.drop(columns_to_drop, axis=1, inplace=True)

# Filter out Match NaN values
initial_rows = len(df)
df = df[df['Match '].notna()]
filtered_rows = len(df)

# Convert Match column to int

df['Match '] = df['Match '].astype(int)

# Create swapped dataframe for bidirectional analysis
df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

# Concatenate dataframes
df_full = pd.concat([df, df_swap], axis=0)


# Calculate WIN column
df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])
win_stats = df_full['WIN'].value_counts().to_dict()


# Create wins pivot table
df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')


# Create alternative team data for schedule strength calculation
df_alter = df_full.copy()


#

In [27]:

df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']


df_alt_merge = df_full.merge(df_alter, on='Match ', how='inner')


# Calculate alternative WIN rates
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])


# Remove rows where Team1 = Alt_Team (self-comparison)
initial_alt_rows = len(df_alt_merge)
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]
filtered_alt_rows = len(df_alt_merge)


In [28]:
df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

In [29]:
df_alt_wins

Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,4.0,3.0,2.0,0.0,5.0,3.0,3.0,NaN,7.0,8.0,0.0,5.0,0.0,5.0,9.0,11.0,5.0,3.0,2.0,9.0,3.0,13.0,2.0,6.0,6.0,3.0,4.0,0.0,3.0
Boston Celtics,3.0,NaN,5.0,2.0,4.0,1.0,2.0,0.0,2.0,1.0,2.0,0.0,9.0,7.0,3.0,9.0,13.0,2.0,1.0,5.0,1.0,NaN,7.0,9.0,1.0,4.0,12.0,0.0,0.0,1.0
Brooklyn Nets,4.0,3.0,NaN,3.0,2.0,5.0,2.0,6.0,2.0,5.0,4.0,2.0,8.0,5.0,4.0,7.0,8.0,3.0,0.0,7.0,5.0,3.0,9.0,4.0,2.0,5.0,8.0,1.0,1.0,3.0
Charlotte Hornets,4.0,12.0,5.0,NaN,8.0,7.0,6.0,1.0,6.0,3.0,3.0,2.0,16.0,9.0,6.0,11.0,11.0,2.0,3.0,7.0,2.0,1.0,7.0,10.0,2.0,4.0,16.0,0.0,4.0,5.0
Chicago Bulls,2.0,7.0,6.0,4.0,NaN,7.0,3.0,2.0,5.0,0.0,2.0,0.0,9.0,10.0,5.0,7.0,7.0,3.0,2.0,10.0,1.0,2.0,3.0,10.0,1.0,2.0,14.0,2.0,3.0,5.0
Cleveland Cavaliers,1.0,3.0,2.0,0.0,2.0,NaN,0.0,3.0,2.0,0.0,3.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,2.0,6.0,4.0,3.0,3.0,3.0,0.0,1.0,4.0,1.0,2.0,3.0
Dallas Mavericks,6.0,13.0,6.0,6.0,6.0,2.0,NaN,3.0,3.0,6.0,6.0,0.0,12.0,7.0,10.0,12.0,15.0,7.0,2.0,5.0,4.0,1.0,9.0,10.0,5.0,7.0,12.0,4.0,1.0,1.0
Denver Nuggets,6.0,3.0,2.0,2.0,4.0,8.0,2.0,NaN,2.0,7.0,9.0,5.0,3.0,4.0,2.0,3.0,5.0,5.0,6.0,9.0,9.0,7.0,7.0,4.0,6.0,3.0,5.0,7.0,2.0,7.0
Detroit Pistons,NaN,6.0,6.0,4.0,8.0,8.0,3.0,5.0,NaN,1.0,2.0,2.0,9.0,10.0,3.0,5.0,4.0,1.0,3.0,11.0,3.0,3.0,2.0,8.0,1.0,2.0,12.0,2.0,3.0,6.0
